# isak_505_521 (crypto)

Challenge file is provided in [isak_505_521.zip](isak_505_521.zip).

This was one of the challenges released right at the start of the CTF, so in theory we had a whole 24 hours to solve it. I worked on it on and off with JuliaPoo throughout, and by the end of the CTF we were very close to solving it.

This was only my second time encountering a coding theory based cryptosystem, the first time being in WeedSolomon 420 from the Midnight Sun CTF 2022 Quals. Unfortunately, I was complacent enough to think that they wouldn't do a coding theory problem again in the finals, and I couldn't be more wrong.

Fun story: the member of DiceGang that had done a write-up on WeedSolomon 420 (I think it was defund? correct me if not) deleted/privated the gist in the middle of the CTF. I wasn't particularly affected, but another team had the bad luck to be copying something from the gist and then decided to refresh halfway and it was gone.

Anyway, the setup of this challenge is a [McEliece cryptosystem](https://en.wikipedia.org/wiki/McEliece_cryptosystem) using [Reed-Solomon codes](https://en.wikipedia.org/wiki/Reed%E2%80%93Solomon_error_correction). Specifically, we construct the matrix $k \times n$ matrix $\hat{G} = SGP$, where $S$ is a random $k \times k$ non-singular matrix, $G$ is the $n \times k$ generator matrix for a generalised Reed-Solomon code with some "random" columns at the start, and $p$ is a random $n \times n$ permutation matrix. For the purpose of this challenge, we have $k = 202$, $n = 505$, and we work on the field $GF(521)$.

Since $P$ permutes the columns of $G$, it is easier to work directly with the permuted matrix $G' = GP$ which consists of columns that are either of the form $(1, x, x^2, ... x^{k-1})$ for some $x$, or a random column.

It becomes clear then, that the attack decomposes naturally into two stages. In the first part, we have to identify which columns are the Reed-Solomon ones, and remove the others. In the second part, we need to figure out the $\alpha$ corresponding to the code.

## Step 1: Distinguishing Columns

We are given the construction of $G$, so we know that 404 of the 505 columns are "good" (i.e. part of the Reed-Solomon code), while the remaining 101 are "bad" (i.e. circulant, but we don't use this information).

We use a trick here from [Gauthier et al.](https://arxiv.org/abs/1204.6459), which is to construct the square code of $\hat{G}$, which roughly speaking is the matrix consisting of all elementwise-products of pairs of rows of $G$, which would have dimensions $\binom{505}{2} \times {505}$. This has rank at most 505, but the thing to note here is that the 101 random columns have full rank whereas the 404 Reed-Solomon columns are linearly dependent. Why is this the case? When you multiply a row of $S$ with a column of $G$, you get a polynomial of degree 201. When you multiply two polynomials of degree 201 you get a polynomial of degree 402. This means that the vector space spanned by all the polynomials has dimension (at most) 403, and so they're linearly dependent.

How do we use this property? We can find the reduced row-echelon form of the matrix, and there will two types of rows. The Reed-Solomon ones will span multiple columns while the random ones will span a single column.

Furthermore, we can optimise this operation by not calculating every possible row of the square code. Since it has rank less than 505 anyway, we only really need to compute 505 or so rows. For our purposes, we take all pairs of the first 33 rows of $G$.

In [1]:
Gpub = load('Gpub')
c2 = load('c2')

k, n = Gpub.dimensions() # 202x505
F = c2[0].parent()       # GF(521)

# construct the square code (33 rows suffices because 33C2 > 505; doing all pairs still works but takes longer)
foo =  matrix([a.pairwise_product(b) for a,b in Combinations(Gpub[:33],2)])
foo.echelonize()
assert foo.rank() == n - 1

# find all random (i.e. circulant) columns
poss = []
for v in foo:
    nzp = v.nonzero_positions()
    if not nzp: break
    if len(nzp) == 1: poss.append(nzp[0])
print(f'Found {len(poss)} bad columns: {poss = }')

Found 101 bad columns: poss = [0, 1, 2, 5, 12, 15, 26, 28, 29, 30, 31, 34, 35, 46, 47, 52, 54, 56, 59, 75, 76, 77, 78, 90, 97, 99, 103, 109, 112, 118, 124, 126, 127, 132, 137, 140, 145, 149, 158, 159, 163, 169, 181, 186, 199, 203, 204, 207, 209, 211, 216, 221, 222, 234, 237, 242, 247, 253, 261, 266, 271, 273, 280, 283, 286, 291, 297, 301, 313, 320, 345, 354, 355, 366, 367, 369, 378, 380, 393, 401, 402, 406, 408, 419, 423, 427, 444, 445, 460, 462, 463, 469, 471, 473, 475, 477, 481, 483, 486, 496, 504]


Success! We've found all 101 bad columns, so we can remove these from both $\hat{G}$ and the ciphertext.

In [2]:
b = Gpub.delete_columns(poss)
ct = matrix(c2).delete_columns(poss)[0]

## Step 2: Finding the $\alpha$ that makes up $G'$

We have removed all bad columns, but we still need to figure out how $\hat{G}$ decomposes into $SG$. We do know that the decomposition is not unique -- we can take any linear transformation of $\alpha$ and still have a valid decomposition (though with a different $S$). As such, we can fix $\alpha_0 = 0$ and $\alpha_1 = 1$ which should give us a unique solution.

We follow the paper (lecture notes in this case) by [Sidelnikov-Shestakov](https://crypto-kantiana.com/elena.kirshanova/talks/Sidelnikov_Shestakov.pdf) to derive the rest of $\alpha$. The construction actually lets $\alpha_2 = \infty$ which uniquely determines the rest, but at the expense of working in the projective field $GF(521) \cup \{\infty\}$. Instead, we will work with a single unknown parameter $m$ that will uniquely determine the rest of $\alpha$, which is more or less equivalent, except we will also find the one solution that doesn't require $\infty$.

In [3]:
c0b = b.matrix_from_columns([0]+list(range(k+2,2*k))).left_kernel().basis()[0]*b
c1b = b.matrix_from_columns([1]+list(range(k+2,2*k))).left_kernel().basis()[0]*b
for m in F:
    try:
        test=[0,1]+[m/(m+c1b[j]/c0b[j]) for j in range(2,k+2)]
        S_test = b[:,:k+2]/matrix.vandermonde(test).T[:k]
        alpha=(1/S_test*b)[1]
        print(f'Success at {m = }: {alpha = }')
        break
    except:
        pass

Success at m = 85: alpha = (0, 1, 143, 379, 372, 465, 362, 453, 492, 419, 44, 351, 400, 55, 353, 67, 361, 374, 215, 5, 499, 490, 507, 114, 100, 146, 389, 99, 184, 56, 171, 388, 456, 468, 281, 43, 423, 347, 481, 183, 325, 145, 74, 497, 26, 2, 285, 459, 148, 514, 424, 178, 9, 104, 238, 139, 165, 324, 197, 227, 163, 187, 205, 333, 217, 368, 119, 219, 327, 162, 404, 158, 313, 150, 455, 173, 337, 329, 476, 515, 503, 399, 473, 316, 22, 414, 271, 47, 384, 87, 252, 3, 415, 318, 433, 411, 331, 144, 451, 175, 21, 409, 289, 487, 235, 46, 254, 24, 170, 308, 216, 176, 434, 54, 7, 31, 309, 11, 50, 269, 277, 80, 449, 265, 207, 428, 85, 182, 355, 172, 360, 117, 342, 166, 62, 95, 457, 68, 479, 426, 53, 101, 274, 500, 200, 436, 431, 213, 154, 248, 304, 283, 403, 23, 51, 177, 258, 435, 19, 520, 352, 392, 15, 470, 253, 290, 239, 69, 89, 493, 358, 45, 36, 371, 140, 115, 259, 326, 116, 233, 489, 96, 348, 60, 494, 76, 508, 296, 191, 230, 225, 365, 395, 491, 66, 131, 232, 335, 441, 413, 386, 303, 448, 430, 12

Hooray, we've done it! At this point, it is _probably_ possible to determine the original $\alpha$ from the circulant columns (up to cyclic order), but we don't need that to decode.

In [4]:
C = codes.GeneralizedReedSolomonCode(alpha, k)
D = codes.decoders.GRSBerlekampWelchDecoder(C)
bytes(b.solve_left(D.decode_to_code(matrix(c2).delete_columns(poss)[0])))

b'no_isd_just_build_an_efficient_distinguisher_to_get_the_flag!r_u_schuuuuuur?yassssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss'

Presumably, the flag is just all that wrapped in `midnightsun{}`, but we didn't actually manage to solve it while the CTF was live.